In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the datas
product = pd.read_csv('Data_01.Pruduct.NoADDNoHit.csv', dtype={"CLNT_ID": np.str, "SESS_ID": np.str, "PD_C": np.str}, thousands=",", usecols = [0,1,2,3,5,6])
custom = pd.read_csv('Data_04.Custom.csv',dtype={"CLNT_ID": np.str})
session = pd.read_csv('Data_05.Session.Day.csv',thousands=",", dtype={"CLNT_ID": np.str, "SESS_ID": np.str},usecols=[0,1,3,4,5,6,7])
master = pd.read_csv('Data_06.Master.Col.csv', dtype={"PD_C": np.str},usecols=[0,5])

In [3]:
custom.head()

,CLNT_ID,CLNT_GENDER,CLNT_AGE
0,4830726,F,30
1,4830874,F,40
2,4830975,F,30
3,4831275,F,30
4,4825325,F,30


In [4]:
dummies = pd.get_dummies(custom['CLNT_GENDER'], prefix='GENDER')
custom = pd.concat([custom, dummies], axis=1)
custom.drop("CLNT_GENDER", axis=1, inplace=True)
custom.head()

,CLNT_ID,CLNT_AGE,GENDER_F,GENDER_M
0,4830726,30,1,0
1,4830874,40,1,0
2,4830975,30,1,0
3,4831275,30,1,0
4,4825325,30,1,0


In [8]:
m1 = pd.merge(product, custom, on='CLNT_ID', how = 'inner')

In [5]:
session.dropna(inplace=True)
session['SESS_DT'] = pd.to_datetime(session['SESS_DT'], format="%Y%m%d")
session['SESS_month'] = pd.DatetimeIndex(session['SESS_DT']).month
session.head()

,CLNT_ID,SESS_ID,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,SESS_month
0,5873599,08641867,2018-05-09,82.0,1890.0,mobile,Daejeon,5
1,5873599,06616320,2018-06-11,105.0,1604.0,mobile,Busan,6
2,5873599,05886172,2018-06-24,41.0,632.0,mobile,Daejeon,6
3,5873884,01050889,2018-09-13,160.0,1035.0,mobile,Gyeonggi-do,9
4,5874461,10298270,2018-04-12,13.0,298.0,mobile,Seoul,4


In [6]:
#categorical variable to dummy
categorical_features = ['DVC_CTG_NM', 'ZON_NM', 'SESS_month']
prefix_name = ["DVC","ZON","month"]

for i in range(3):
    dummies = pd.get_dummies(session[categorical_features[i]], prefix=prefix_name[i])
    session = pd.concat([session, dummies], axis=1)
    session.drop(categorical_features[i], axis=1, inplace=True)
session.head()

,CLNT_ID,SESS_ID,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_desktop,DVC_mobile,DVC_tablet,ZON_Busan,ZON_Chungcheongbuk-do,...,ZON_Jeollabuk-do,ZON_Jeollanam-do,ZON_Seoul,ZON_Ulsan,month_4,month_5,month_6,month_7,month_8,month_9
0,5873599,08641867,2018-05-09,82.0,1890.0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5873599,06616320,2018-06-11,105.0,1604.0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,5873599,05886172,2018-06-24,41.0,632.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,5873884,01050889,2018-09-13,160.0,1035.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5874461,10298270,2018-04-12,13.0,298.0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [10]:
m1 = pd.merge(m1, session, on=['CLNT_ID','SESS_ID'], how = 'inner')

In [11]:
labels, uniques = pd.factorize(master['CLAC0_NM'],sort=True)
print(uniques)

Index(['가구/침구/인테리어', '가전/디지털', '반려동물', '스포츠/레저', '식품', '여행/서비스/금융', '유아동',
       '주방/생활/건강', '패션의류', '화장품/뷰티'],
      dtype='object')


In [12]:
dummies = pd.get_dummies(labels, prefix='CLAC0')
master = pd.concat([master, dummies], axis=1)
master.drop("CLAC0_NM", axis=1, inplace=True)
master.head()

,PD_C,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
0,064382,0,0,0,1,0,0,0,0,0,0
1,062282,0,0,0,0,0,0,0,0,1,0
2,061729,0,0,0,0,0,0,1,0,0,0
3,061537,0,0,0,0,0,0,0,0,1,0
4,058820,0,0,0,0,0,0,0,0,1,0


In [13]:
m1 = pd.merge(m1, master, on=['PD_C'], how = 'inner')
m1.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_BUY_AM,PD_BUY_CT,CLNT_AGE,GENDER_F,GENDER_M,SESS_DT,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
0,4140076,10189797,13,570603,39000,2,40,1,0,2018-04-13,...,0,0,0,1,0,0,0,0,0,0
1,4148106,06537318,21,570603,39000,1,30,1,0,2018-06-13,...,0,0,0,1,0,0,0,0,0,0
2,4148106,06662803,94,570603,39000,1,30,1,0,2018-06-11,...,0,0,0,1,0,0,0,0,0,0
3,5298612,04443505,129,570603,39000,1,30,1,0,2018-07-17,...,0,0,0,1,0,0,0,0,0,0
4,5553084,06453463,55,570603,39000,1,30,1,0,2018-06-14,...,0,0,0,1,0,0,0,0,0,0


In [14]:
m1_mean = m1.groupby(['CLNT_ID'])['HITS_SEQ','PD_BUY_AM','PD_BUY_CT','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','CLNT_AGE','GENDER_F','GENDER_M'].agg(['mean'])
m1_mean.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M
,mean,mean,mean,mean,mean,mean,mean,mean
CLNT_ID,,,,,,,,
0000008,57.000000,38000.000000,1.0,46.500000,417.000000,20,0,1
0000014,284.000000,34900.000000,1.0,256.000000,7098.000000,30,1,0
0000032,58.000000,15360.000000,1.0,98.200000,2169.000000,40,0,1
0000056,47.333333,7933.333333,1.0,45.666667,1157.666667,40,1,0
0000058,209.000000,21850.000000,1.0,195.000000,3161.000000,30,1,0


In [15]:
m1_factor = m1.loc[:, m1.columns.str.startswith('DVC') | m1.columns.str.startswith('ZON') | 
                   m1.columns.str.startswith('month') | m1.columns.str.startswith('CLAC0')]
m1_factor.head()

,DVC_desktop,DVC_mobile,DVC_tablet,ZON_Busan,ZON_Chungcheongbuk-do,ZON_Chungcheongnam-do,ZON_Daegu,ZON_Daejeon,ZON_Gangwon-do,ZON_Gwangju,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [16]:
m1_sum = pd.concat([m1['CLNT_ID'], m1_factor], axis=1).groupby(['CLNT_ID']).agg(['sum'])
m1_sum.head()

,DVC_desktop,DVC_mobile,DVC_tablet,ZON_Busan,ZON_Chungcheongbuk-do,ZON_Chungcheongnam-do,ZON_Daegu,ZON_Daejeon,ZON_Gangwon-do,ZON_Gwangju,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
CLNT_ID,,,,,,,,,,,,,,,,,,,,,
0000008,0.0,2.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000014,0.0,2.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000032,0.0,5.0,0,5.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0.0,0,0,0.0,5.0,0.0
0000056,0.0,3.0,0,3.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0.0,0,1,0.0,0.0,2.0
0000058,0.0,6.0,0,6.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,1.0,0.0,0,0,0.0,5.0,0.0


In [17]:
m2 = pd.concat([m1_mean, m1_sum ], axis=1)
m2.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M,DVC_desktop,DVC_mobile,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
,mean,mean,mean,mean,mean,mean,mean,mean,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
CLNT_ID,,,,,,,,,,,,,,,,,,,,,
0000008,57.000000,38000.000000,1.0,46.500000,417.000000,20,0,1,0.0,2.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000014,284.000000,34900.000000,1.0,256.000000,7098.000000,30,1,0,0.0,2.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000032,58.000000,15360.000000,1.0,98.200000,2169.000000,40,0,1,0.0,5.0,...,0,0,0,0.0,0.0,0,0,0.0,5.0,0.0
0000056,47.333333,7933.333333,1.0,45.666667,1157.666667,40,1,0,0.0,3.0,...,0,0,0,0.0,0.0,0,1,0.0,0.0,2.0
0000058,209.000000,21850.000000,1.0,195.000000,3161.000000,30,1,0,0.0,6.0,...,0,0,0,1.0,0.0,0,0,0.0,5.0,0.0


In [18]:
m2.columns = m2.columns.get_level_values(0)
m2.head()

,HITS_SEQ,PD_BUY_AM,PD_BUY_CT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLNT_AGE,GENDER_F,GENDER_M,DVC_desktop,DVC_mobile,...,CLAC0_0,CLAC0_1,CLAC0_2,CLAC0_3,CLAC0_4,CLAC0_5,CLAC0_6,CLAC0_7,CLAC0_8,CLAC0_9
CLNT_ID,,,,,,,,,,,,,,,,,,,,,
0000008,57.000000,38000.000000,1.0,46.500000,417.000000,20,0,1,0.0,2.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000014,284.000000,34900.000000,1.0,256.000000,7098.000000,30,1,0,0.0,2.0,...,0,0,0,0.0,0.0,0,0,0.0,2.0,0.0
0000032,58.000000,15360.000000,1.0,98.200000,2169.000000,40,0,1,0.0,5.0,...,0,0,0,0.0,0.0,0,0,0.0,5.0,0.0
0000056,47.333333,7933.333333,1.0,45.666667,1157.666667,40,1,0,0.0,3.0,...,0,0,0,0.0,0.0,0,1,0.0,0.0,2.0
0000058,209.000000,21850.000000,1.0,195.000000,3161.000000,30,1,0,0.0,6.0,...,0,0,0,1.0,0.0,0,0,0.0,5.0,0.0


In [20]:
m2.shape

(671470, 43)

In [19]:
m2.to_csv('data_cluster.csv',index=False , encoding='utf-8')